In [1]:
!pip install deep-river

In [2]:
import deep_river as dr
print(dir(dr))

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'anomaly', 'base', 'classification', 'regression', 'utils']


In [3]:
!pip install river

In [4]:
import numpy as np

In [7]:
from river import metrics, datasets, preprocessing, compose
from deep_river import classification
from torch import nn
from torch import optim
from torch import manual_seed

_ = manual_seed(42)

class MyModule(nn.Module):
    def __init__(self, n_features):
        super(MyModule, self).__init__()
        self.dense0 = nn.Linear(n_features, 5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.softmax(X)
        return X

model_pipeline = compose.Pipeline(
    preprocessing.StandardScaler(),
    classification.Classifier(module=MyModule, loss_fn='binary_cross_entropy', optimizer_fn='adam')
)

dataset = datasets.Phishing()
metric = metrics.Accuracy()

for x, y in dataset:
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get():.4f}")


Accuracy: 0.7264


In [9]:
from deep_river.anomaly import Autoencoder
from river import metrics
from river.datasets import CreditCard
from torch import nn
import math
from river.compose import Pipeline
from river.preprocessing import MinMaxScaler

dataset = CreditCard().take(5000)
metric = metrics.ROCAUC(n_thresholds=50)

class MyAutoEncoder(nn.Module):
    def __init__(self, n_features, latent_dim=3):
        super(MyAutoEncoder, self).__init__()
        self.linear1 = nn.Linear(n_features, latent_dim)
        self.nonlin = nn.LeakyReLU()
        self.linear2 = nn.Linear(latent_dim, n_features)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X, **kwargs):
        X = self.linear1(X)
        X = self.nonlin(X)
        X = self.linear2(X)
        return self.sigmoid(X)

ae = Autoencoder(module=MyAutoEncoder, lr=0.005)
scaler = MinMaxScaler()
model = Pipeline(scaler, ae)

for x, y in dataset:
    score=model.score_one(x)
    model.learn_one(x=x)
    metric.update(y, score)

print(f"ROCAUC: {metric.get():.4f}")



Uncompressing into /root/river_data/CreditCard
ROCAUC: 0.7812
